In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dados.csv')

def returnGender(g):
    if(g == 1):
        return "Masculino"
    else:
        return "Feminino"
    
def returnUF(u):
    return {
        11: "Rondônia",
        12: "Acre",
        13: "Amazonas",
        14: "Roraima",
        15: "Pará",
        16: "Amapá",
        17: "Tocantis",
        21: "Maranhão",
        22: "Piauí",
        23: "Ceará",
        24: "Rio Grande do Norte",
        25: "Paraíba",
        26: "Pernambuco",
        27: "Alagoas",
        28: "Sergipe",
        29: "Bahia",
        31: "Minas Gerais",
        32: "Espírito Santo",
        33: "Rio de Janeiro",
        35: "São Paulo",
        41: "Paraná",
        42: "Santa Catarina",
        43: "Rio Grande do Sul",
        50: "Mato Grosso do Sul",
        51: "Mato Grosso",
        52: "Goias",
        53: "Distrito Federal"
    }[u]

def returnEthnicGroup(e):
    return {
        0: "Indígena",
        2: "Branco",
        4: "Negro",
        6: "Asiático",
        8: "Pardo",
        9: "Sem Declaração"
    }[e]

def returnGruposDeIdade(i):
    if(i <= 18):
        return "Menor de 18 anos"
    elif(i <= 25):
        return "Entre 18 e 25 anos"
    elif(i <= 40):
        return "Entre 25 e 40 anos"
    elif (i <= 60):
        return "Entre 40 e 60 anos"
    else:
        return "Maior de 60 anos"

def returnAnosDeEstudo(e):
    if(e == 17):
        return "Não Informado"
    return e - 1

def returnClasseSocial(r):
    salario_minimo = 1100
    if(r <= salario_minimo):
        return "E"
    elif(r <= salario_minimo*3):
        return "D"
    elif(r <= salario_minimo*5):
        return "C"
    elif(r <= salario_minimo*15):
        return "B"
    else:
        return "A"


df['Sexo'] = df['Sexo'].apply(lambda x: returnGender(x))
df['UF'] = df['UF'].apply(lambda x: returnUF(x))
df['Cor'] = df['Cor'].apply(lambda x: returnEthnicGroup(x))
df['Idade'] = df['Idade'].apply(lambda x: returnGruposDeIdade(x))
df['Anos de Estudo'] = df['Anos de Estudo'].apply(lambda x: returnAnosDeEstudo(x))
df['Renda'] = df['Renda'].apply(lambda x: returnClasseSocial(x))

In [6]:
def calcularEntropia(df, coluna):
    values = df[coluna].unique()
    entropia = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        entropia += pi*np.log2(pi)
        
    return -entropia
    
def calcularGanho(df, resultado, coluna):
    values = df[coluna].unique()
    ganho = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        ganho = pi * calcularEntropia(df[df[coluna].eq(i)], resultado)
    ganho = calcularEntropia(df, resultado) - ganho
    return ganho

In [7]:
class No:
    def __init__(self, atributo, galhos):
        self.atributo = atributo
        self.galhos = galhos
        
def construirArvoreDeDecisao(df, colunas, resultado, resultadoAnterior=0, dfOriginal=df):
    if(len(df[resultado]) == 0):
        return No(resultadoAnterior, '.')
    elif(len(df[resultado].unique()) == 1):
        return No(df[resultado].iloc[0], '.')
    elif(len(colunas) == 0):
        return No(df[resultado].value_counts()[:1].index[0], '.')
    
    maiorGanhoDeColunas = -np.inf
    for i in colunas:
        colunaEscolhida = i if maiorGanhoDeColunas < calcularGanho(df, resultado, i) else maiorGanhoDeColunas
    
    galhos = {}
    for i in dfOriginal[colunaEscolhida].unique():
        galhos[i] = construirArvoreDeDecisao(df[df[colunaEscolhida].eq(i)], list(set(colunas) - set([colunaEscolhida])), resultado, df[resultado].value_counts().index[0], dfOriginal)
        
    return No(colunaEscolhida, galhos)


arvore = construirArvoreDeDecisao(df, ['UF', 'Sexo', 'Idade', 'Cor', 'Anos de Estudo'], 'Renda')

In [8]:
def buscarNaArvore(arvore, individuo):
    if(arvore.galhos == '.'):
        return arvore.atributo
    else:
        return buscarNaArvore(arvore.galhos[individuo[arvore.atributo]], individuo)

buscarNaArvore(arvore, {
    'UF': 'Rio de Janeiro',
    'Sexo': 'Masculino',
    'Idade': 'Entre 40 e 60 anos',
    'Cor': 'Branco',
    'Anos de Estudo': 14
})

'E'

E    5
D    3
B    2
C    2
Name: Renda, dtype: int64